## Initialization

In [7]:
storageName = 'university'

if storageName == 'paperspace': 
    guy_folder = "/notebooks/"
elif storageName == 'colab':
    guy_folder = "/content/"
elif storageName == 'university':
    guy_folder = '/vol/scratch/guy/'
    
    
cache_dir = guy_folder+"/cache/transformer_cache"

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import transformers
from transformers import AutoModel, AutoTokenizer
from transformers import AdamW
%matplotlib inline

## Functions

In [76]:
def _introduceBinLinear(m_):
    children = m_.named_children()
    for name, m in children:
        if name in ['k_lin', 'q_lin', 'v_lin', 'out_lin', 'lin1', 'lin2']:
            m_.__setattr__(name, BinLinear(m.weight, m.bias))

In [79]:
class Binarizer(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        threshold = Binarizer.threshold
        output = (input > threshold).float()        
        
        return output

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

    @staticmethod
    def setThreshold(threshold):
        Binarizer.threshold = threshold
    

    
Binarizer.setThreshold(0.5)


class BinLinear(nn.Module):
    def __init__(self, originalWeight, originalBias, initialSparsity = .7):
        # Notice positivity of the mask is not warranated
        super().__init__()
        shape = originalWeight.shape
        self.originalWeight = originalWeight
        self.originalBias = originalBias

        self.rawMask = nn.Parameter(torch.rand(*shape) * self._sparsityMask(initialSparsity, shape))
    
    def forward(self, x):
        w = self.originalWeight * Binarizer.apply(self.rawMask)
        output = x @ (w.t()) 
        if self.originalBias is not None:
            output += self.originalBias
        return output
    
    def _sparsityMask(self, initialSparsity, shape):
        return (torch.rand(*shape) > initialSparsity).float()
    
    def extra_repr(self):
        rep = '{}, {}'.format(*self.rawMask.t().shape)
        if self.originalBias is not None:
            rep += ', bias = True'
        return rep

## Main

In [103]:
nn.Linear??

In [108]:
class BinBert(nn.Module):
    def __init__(self, n_classes = None):
        super().__init__()
        self.body = AutoModel.from_pretrained("distilbert-base-cased", cache_dir = cache_dir)
        self.body.requires_grad_(False)
        self.body.apply(_introduceBinLinear);
        if n_classes is not None:
            shape = (768, n_classes)
            self.pooler = BinLinear(nn.Parameter(torch.rand(*shape).t()),
                                    nn.Parameter(torch.rand(shape[1])))
    def forward(self, x):
        output, = self.body(x)
        if hasattr(self, 'pooler'):
            output = self.pooler(output)
        return output

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 
 the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said 
 <unk> inc. the unit of new york-based <unk> corp. that makes kent cigarettes stopped using <unk> in its <unk> cigarette filters in N 
 although preliminary findings wer

In [105]:
n_classes = 
binBert = BinBert(n_classes)

In [106]:
binBert(torch.zeros(1, 512).long())

tensor([[[ 0.8820,  1.0757, -0.1496,  ...,  1.1517,  1.2714,  0.8045],
         [-1.4825,  2.0222,  1.0192,  ...,  0.7261, -1.6243, -1.2302],
         [-1.4870,  1.8818,  0.7808,  ...,  0.0284, -1.1162,  1.3303],
         ...,
         [-1.4008,  0.7714,  0.2485,  ...,  1.3958, -0.4883,  1.6845],
         [-0.5316,  1.3735,  1.2314,  ...,  2.1108,  0.7832,  2.8921],
         [-2.3742,  0.2228,  1.4076,  ...,  3.0544,  0.2483,  1.4370]]],
       grad_fn=<AddBackward0>)